# ThoroughBet Simulation


## Load necessary modules

In [1]:
import numpy as np
import pandas as pd

## Load data

In [2]:
df = pd.read_csv('racehorse_data.csv', index_col=[0])
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f54,f55,f56,f57,runner_id,result,event_id,is1,is2,oos
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,...,0.040241,-0.003450,-0.003327,0.036781,353046,3,287398,True,True,False
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,...,0.179168,-0.002913,-0.002791,0.213007,352217,5,287398,True,True,False
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,...,0.217412,-0.012249,-0.012108,-0.173009,342962,13,287398,True,True,False
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,...,0.399248,0.009783,0.009877,0.326784,361432,1,287398,True,True,False
4,0.064913,1.343377,0.242782,0.008894,0.0007,-0.017472,-0.199782,0.004808,0.481565,1.404777,...,-0.325201,0.004683,0.004789,-0.145831,359420,6,287398,True,True,False


In [3]:
from itertools import product
def set_parwise(ar):
    one = np.array(np.random.choice(ar))
    ar_one = np.setdiff1d(ar, one)
    one_ar = np.repeat(one, len(ar_one))
    return [x for x in zip(one_ar, ar_one)]

In [4]:
def df_cut(data, result):
    
    event = np.intersect1d(data[data.result == result-1]['event_id'], data[data.result == result]['event_id'])
    mask = np.in1d(data['event_id'],event)
    return data[mask]

In [5]:
col_names = df.columns[:-6].tolist()+['event_id', 'result']

In [6]:
def df_diff (data, results):
    
    first, second = results
    if not type(data) is np.ndarray:
        X = data.values
    mask1 = np.in1d(X[:,-1], first)
    mask2 = np.in1d(X[:,-1], second)
    X_diff = X[mask1] - X[mask2]
    rand = np.random.choice([1,-1], len(X_diff))
    return X_diff*rand.T

- парное ранжирование для мест 1и 2, 2и 3, 3 и 4

In [7]:
def ranking_data(df, list_result =[2, 3, 4], 
                columns = ['f{}'.format(i) for i in range(1,58)] +['result','event_id']):
    
    df_event = pd.Series()
    
    X_data = np.zeros((0,len(columns)-1))
    for n in list_result:
        print ''
        df_r = df_cut(df, n) # only data where are first and second place
        first , second = n-1, n # place 1, 2 
        # data where first place
        X1 = df_r[columns][df_r['result'] == first].drop_duplicates(subset ='event_id').values
        # events which use in data
        df_event = df_event.append(df_r['event_id'][df_r['result'] == first])
        #print df_event.shape
        # data where second place
        X2 = df_r[columns][df_r['result'] == second].drop_duplicates(subset ='event_id').values
        # differance between first and second place mix of the order  from "first -second" or "second -first"
        X = (X1 -X2)* np.random.choice([1,-1], len(X1)).reshape(-1,1) 
        X_data = np.vstack((X_data, X[:, :-1])) # union of set of deta matrix
        #print X.shape, X_data.shape
        mask_class = np.in1d(X_data[:,-1],[-1.,1.]) # only two class "-1, 1"
    return X_data[mask_class], df_event

In [8]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

In [9]:
def clf_llhood (data, clf, result =[2,3,4], 
               columns = ['f{}'.format(i) for i in range(1,58)] +['result','event_id']):
    
    from sklearn.metrics import accuracy_score
    #print np.sum(data.is1), result
    X_y_data, _ = ranking_data(data[data.is1], result, columns)
    X_train, y_train = X_y_data[:,:-1], X_y_data[:,-1]
    clf.fit(X_train, y_train)
    print clf
    X_y_data, _ = ranking_data(data[data.oos], result, columns)
    X_test, y_test = X_y_data[:,:-1], X_y_data[:,-1]
    print 'accuracy  test', accuracy_score ( y_test, clf.predict(X_test))
    print
    factors = columns[:-2]
    try:
        data['S_%s'%str(clf)[:6]] = clf.decision_function(data[factors].values)
        data['p_%s'%str(clf)[:6]] = data['S_%s'%str(clf)[:6]].groupby(by = data.event_id).apply(softmax)
    except:
        data['p_%s'%str(clf)[:6]] =data[factors].groupby(by = data.event_id).apply(lambda x: P_win_from_pair(x, clf))
    
    data['log_p_%s'%str(clf)[:6]] = np.log(data['p_%s'%str(clf)[:6]])
    print 'is1  mean ll %s'%str(clf)[:6], data.ix[data.is1 & (data.result ==1),'log_p_%s'%str(clf)[:6]].mean()*1000
    print 
    print 'oos  mean ll %s'%str(clf)[:6], data.ix[data.oos & (data.result ==1),'log_p_%s'%str(clf)[:6]].mean()*1000
    return data, clf

In [10]:
def clustering(cluster, X_train, y_train):
    
    X_tr, y_tr = {}, {}
    for cl in np.unique(cluster):
        mask = np.in1d(cluster, cl)
        X_tr[cl] = X_train[mask]
        y_tr[cl] = y_train[mask]
    return X_tr, y_tr

In [11]:
def new_features(X, X_pca, rank=2):
    
    for n in range(rank):
        X_ = X *X_pca[:,n].reshape(-1,1)
        if n ==0:
            X_new = np.hstack((X, X_))
        else:
            X_new = np.hstack((X_new, X_))
    return X_new

In [12]:
def PCA_features(X, pca):
    
    try:
        pca.noise_variance_
        X_pca = pca.transform(X)
    except AttributeError:
        X_pca = pca.fit_transform(X)
    rank = pca.n_components_
    X_new = new_features(X, X_pca, rank)
    return X_new, pca

- доболнительные признаки add

In [13]:
def mean_max_min(data):
    
    n, m = data.shape
    max_min = np.zeros((n,m))
    index  = data.index
    #print index
    for i,j in enumerate(index):
        #print np.setdiff1d(index, i)
        data_ = data.ix[np.setdiff1d(index, j), :].values
        #print data_[:2]
        max_min[i, :] = (data_.max(axis =0) - data_.min(axis =0))/n
        
    return pd.DataFrame(data = max_min, columns = data.columns)

In [14]:
df_f = df.ix[:,u'f1':u'f57'].groupby(by = df['event_id']).apply(mean_max_min)
df_f['is1'] = df.ix[:,u'is1'].groupby(by =df['event_id']).first()
df_f['oos'] = df.ix[:,u'oos'].groupby(by =df['event_id']).first()
for col in df_f.columns[:-2]:
    df[str('M_'+col)]= df_f[col].values
df.head(10)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,M_f48,M_f49,M_f50,M_f51,M_f52,M_f53,M_f54,M_f55,M_f56,M_f57
0,0.064913,1.395654,-0.152068,0.008172,0.000700,0.971937,1.096287,-0.003309,1.761018,0.968297,...,0.007494,0.064775,0.107379,0.002549,0.069281,0.092629,0.056297,0.001695,0.001691,0.058936
1,-0.250759,1.137530,0.019747,0.008894,0.000700,0.398019,0.481980,-0.002772,1.682896,0.354919,...,0.005310,0.064775,0.107379,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936
2,-0.250759,1.029570,-0.145221,-0.020776,0.000700,0.307072,0.427015,-0.012089,1.165470,0.440719,...,0.007494,0.057598,0.107379,0.002549,0.069281,0.097933,0.056297,0.001415,0.001412,0.058936
3,0.064913,1.395654,-0.136884,-0.020776,0.000700,0.234280,0.361007,0.009897,0.880806,1.230350,...,0.007494,0.064775,0.107379,0.002549,0.069281,0.097933,0.042310,0.001533,0.001530,0.050184
4,0.064913,1.343377,0.242782,0.008894,0.000700,-0.017472,-0.199782,0.004808,0.481565,1.404777,...,0.007494,0.064775,0.084504,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936
5,0.182570,1.029570,0.201410,0.003267,-0.002216,0.023445,-0.225265,0.001317,0.162080,0.121588,...,0.007494,0.064775,0.107379,0.002549,0.055455,0.097933,0.056297,0.001695,0.001691,0.058936
6,0.064913,0.508767,-0.080697,0.002984,-0.000843,0.130482,-0.225265,-0.002333,0.032518,-0.057125,...,0.007494,0.064775,0.107379,0.002517,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936
7,0.182570,0.336260,-0.055067,0.001173,-0.003430,0.044895,-0.225265,-0.003962,-0.033998,0.032210,...,0.007059,0.064741,0.107379,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936
8,-0.114914,0.508767,-0.100332,0.010256,0.000700,0.260870,0.341749,0.001081,0.481565,-0.507366,...,0.007494,0.064775,0.107379,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936
9,-0.023268,0.336260,0.033493,-0.001754,0.000191,-0.160113,-0.357380,0.000226,-0.110449,-0.385231,...,0.007494,0.064775,0.107379,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936


In [15]:
col_names = ['f%s'%i for i in range(1,58)] + ['M_f%s'%i for i in range(1,58)] +['result', 'event_id']

In [16]:
X_y_data, df_event = ranking_data(df[df.is1], [2], col_names)
X_train, y_train = X_y_data[:,:-1], X_y_data[:,-1]
X_y_data, df_event = ranking_data(df[df.oos], [2], col_names)
X_test, y_test = X_y_data[:,:-1], X_y_data[:,-1]

In [17]:
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,M_f48,M_f49,M_f50,M_f51,M_f52,M_f53,M_f54,M_f55,M_f56,M_f57
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,...,0.007494,0.064775,0.107379,0.002549,0.069281,0.092629,0.056297,0.001695,0.001691,0.058936
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,...,0.005310,0.064775,0.107379,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,...,0.007494,0.057598,0.107379,0.002549,0.069281,0.097933,0.056297,0.001415,0.001412,0.058936
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,...,0.007494,0.064775,0.107379,0.002549,0.069281,0.097933,0.042310,0.001533,0.001530,0.050184
4,0.064913,1.343377,0.242782,0.008894,0.0007,-0.017472,-0.199782,0.004808,0.481565,1.404777,...,0.007494,0.064775,0.084504,0.002549,0.069281,0.097933,0.056297,0.001695,0.001691,0.058936


In [18]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators =100, oob_score=True, n_jobs =-1)
forest.fit(X_train, y_train)
print 'train score  ',forest.oob_score_
print 'accuracy  ', accuracy_score ( y_test, forest.predict(X_test))

train score   0.552586972314
accuracy   0.578704720088


In [19]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train, y_train)
print 'train score  ',LDA.score(X_train, y_train)
print 'accuracy  ', accuracy_score ( y_test, LDA.predict(X_test))

train score   0.589313100331
accuracy   0.582217343578


C:\Users\User\Anaconda2\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [20]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
QDA = QuadraticDiscriminantAnalysis()
QDA.fit(X_train, y_train)
print 'train score  ',QDA.score(X_train, y_train)
print 'accuracy  ', accuracy_score ( y_test, QDA.predict(X_test))

train score   0.591573169747
accuracy   0.511964873765


C:\Users\User\Anaconda2\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [21]:
def P_pair(data, clf):
    
    
    n,m = data.shape
    p_matrix = np.zeros((n,n))
    np.fill_diagonal(p_matrix, 0.5) 
    featurs = data.columns[:-2]
    for  i,i_ in  enumerate(data.index):
    
        ar_S_ji = (data.ix[data.index > i_, featurs] - data.ix[i_, featurs]).values
        if ar_S_ji.shape[0] >=1:
            p_matrix[i,i+1:] = forest.predict_proba(ar_S_ji)[:,0].reshape(1,-1)
    
    return np.triu(1 -p_matrix, 1).T + p_matrix

In [22]:
P_win = lambda x: 1/(1/x -1).sum(axis =1)
P_win_from_pair1 = lambda x: P_win(P_pair(x, forest))#.reshape(-1,1)
P_win_from_pair1(df.ix[df.event_id == 287398, col_names])

array([ 0.04252084,  0.05152016,  0.06530585,  0.04005717,  0.04136367,
        0.0651916 ,  0.06061875,  0.06873915,  0.08411466,  0.08938966,
        0.13114909,  0.13640713,  0.1021013 ])

In [23]:
def P_win_from_pair(data, clf):
    n,m = data.shape
    p_matrix = np.zeros((n,n))
    np.fill_diagonal(p_matrix, 0.5)
    features = data.columns
    #print features
    for  i,i_ in  enumerate(data.index):
    
        ar_S_ji = (data.ix[data.index > i_, features] - data.ix[i_, features]).values
        if ar_S_ji.shape[0] >=1:
            p_matrix[i,i+1:] = forest.predict_proba(ar_S_ji)[:,0].reshape(1,-1)
    
    P_pair = np.triu(1 -p_matrix, 1).T + p_matrix
    #print (1/(1/P_pair -1).sum(axis =1))
    return (1/(1/P_pair -1).sum(axis =1))

In [24]:
%%time
df_group = df.ix[:99,col_names[:-2]].groupby(by = df.event_id).apply(lambda x: P_win_from_pair(x, forest))
P_forest = np.array([])
for event in df_group.index:
    P_forest = np.hstack((P_forest,df_group[event]))
#df['p_forest'] = P_forest

Wall time: 27.4 s


In [27]:
np.sum(df.oos)/99.*28

13586.505050505049

In [ ]:
%%time
df_group = df.ix[:,col_names[:-2]].groupby(by = df.event_id).apply(lambda x: P_win_from_pair(x, forest))
P_forest = np.array([])
for event in df_group.index:
    P_forest = np.hstack((P_forest,df_group[event]))
df['p_forest'] = P_forest

In [ ]:
df.head()

In [ ]:
print 'oos  mean ll  ', np.log(df.ix[df.oos & (df.result ==1),'p_forest']).mean()*1000

In [ ]:
from itertools import combinations, combinations_with_replacement, permutations

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logit = LogisticRegression (C =10., penalty ='l1', n_jobs =-1)
df1, logit = clf_llhood(df, logit, [2,3, 4], col_names)




LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)



accuracy  test 0.571049332645

is1  mean ll Logist -2532.7164248

oos  mean ll Logist -2574.31632632


In [43]:
np.abs(logit.coef_).argsort()

array([[ 61, 112, 111, 108,  55,  70,  73,  76,  80,  82,  87,  92,  64,
         95,  16,  12,  30,   1,  56,  28,  41,  42,  45,  57,  23,  32,
          5,  22,   9,  21,  86,  51,  50,  14,  38,  39,  58,  81,  66,
         17,  96,  48,  11,  25,  35,  10,   7,   4,  26,  31,  18,  13,
         40,  24,   6,  98,  52,  53,  34,  97,  20,   8,   0,  15, 106,
         71,  49,  67,  37, 107,  99,  46,  29,  65,  27,  72, 104,  68,
         43,  47,  69,  62,  19, 109,   3,  33,  54,   2,  36,  63, 102,
         84,  79,  60,  78, 105,  89,  94, 103,  75,  85,  83,  93,  44,
         90, 100,  88, 110, 113,  59,  74,  77,  91, 101]], dtype=int64)

In [81]:
[i for i,f in enumerate((np.abs(logit.coef_) == 0.0)[0,:]) if f]

[16, 55, 61, 64, 70, 73, 76, 80, 82, 87, 92, 95, 108, 111, 112]

In [72]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C = 10.0, dual =False, penalty='l1')
df1,svm = clf_llhood(df, svm, [2, 3, 4], col_names)




LinearSVC(C=10.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0)



accuracy  test 0.570606887398

is1  mean ll Linear -2371.90506774

oos  mean ll Linear -2407.64531481


In [73]:
np.abs(svm.coef_).argsort()

array([[111,  57,   1,  87,  12,  56,  30,  41,  42,  28,  64,  95,  96,
         22,  32,  45,  23,  14,   5,   9,  21,  66,  50,  39,  51,  58,
         92,  38,  48,  17,  11,  80,  10,  35,  40,  31,  55,  26,  16,
         25,   4,  70,  18, 108,  13,  24,  98,  97,  53,  52,   6,  81,
         76,   8,  20,  54,   0,  86,  15,   7,  49,  37, 106,  99,  65,
         67,  46,  29,  27,  71,  68, 107,  47,  43,  34,  19,  62,  72,
         69, 109, 102,   3,  61, 104,  33,   2,  79,  63,  36,  89,  78,
         83,  94, 103,  60, 105,  84,  44,  85,  75, 110,  90, 100,  88,
         93, 113,  59,  74,  73, 112,  77,  82, 101,  91]], dtype=int64)

In [82]:
[i for i,f in enumerate((np.abs(svm.coef_) == 0.0)[0,:]) if f]

[57, 111]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators =200, max_depth =2, loss ='exponential')
df1, gbc = clf_llhood(df, gbc, [2, 3, 4], col_names)

In [56]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100)
df1, abc = clf_llhood(df, abc, [2], col_names)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

accuracy  test 0.578265642151

is1  mean ll AdaBoo -2256.8277632

oos  mean ll AdaBoo -2287.55956968


In [62]:
abc.feature_importances_.argsort()

array([ 30,  62,  31,  34,  81,  80,  79,  38,  77,  76,  41,  42,  73,
        71,  70,  48,  50,  51,  66,  55, 112,  63,  58,  59,  85,  28,
        61,  17, 103, 107, 100,  26,  98,  97,  16,  95, 104,  20,  93,
         4, 109, 110,  91,  88, 111,  94, 105, 108,  64,  67,  86,  74,
       102, 101,  75,  96,  82,  84,  92,  87,  69,  56, 113,   8,  22,
        18,  14,  60,  25,  27,  29,  13,  12,  35,   9,  21,  39,  19,
        43,  54,   2,  53,  52,   3,  40,   5,   6,   7,  44,  47,  99,
       106,  15,  10,  57,  24,  90,  65,  68,  49,  72,  23,   0,  33,
        83,  89,  36,  37,  11,  45,  46,   1,  78,  32], dtype=int64)

In [63]:
abc.feature_importances_[30], abc.feature_importances_[32]

(0.0, 0.040000000000000001)

In [61]:
gbc.feature_importances_.argsort()

array([ 79,  55,  91, 101,  64,  10,  16,  34,  63,  38,  73,  68, 104,
        85, 111, 102,  84,  18, 108,  94,  95,  74,  54,  92,  30,  88,
         4,  81,  76,  75,  97,  71,  60,  45,  70, 105,  24, 112,   7,
        86,  40,  61,  58,  48,  57,  36,  98,  35,  72, 109, 106,  62,
        50,   5,  93,  33,  83,  78,  28,  23,  26,  82,  22, 100,   6,
        89,  69,  41, 107, 110,  67,  80,  96,  90, 103,  87,   8,  12,
        59,  11,  17, 113,  21,  77,   2,  52,  42,  44,  53,   0,  51,
        46,  56,   9,  47,  13,  66,  14,  65,  43,  20,   3,  27,  99,
        31,  15,  19,  25,  29,  32,  49,  39,  37,   1], dtype=int64)

In [52]:
df1 = clf_llhood(df, LDA, [2], col_names)


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

accuracy  test 0.582217343578

is1  mean ll Linear -2582.48453657

oos  mean ll Linear -2628.42004762


In [53]:
df1 = clf_llhood(df, QDA, [2], col_names)


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariances=False, tol=0.0001)

accuracy  test 0.501866081229

is1  mean ll Quadra -inf

oos  mean ll Quadra -9045.99619344


In [ ]:
ind = np.random.randint(len(X_train), size=(12,1000))
logits = []

In [25]:
for i in range(len(ind)):
    print logit.fit(X_train[ind[i]], y_train[ind[i]])
    logits.append(logit)
    print 'train  score  ',logit.score(X_train[ind[i]], y_train[ind[i]])
    print 'test Score  ', accuracy_score(y_test, logit.predict(X_test))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
train  score   0.629
test Score   0.566630076839
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
train  score   0.619
test Score   0.56882546652
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
train  score   0.607
test Score   0.572338090011
LogisticRegression(C=1.0, class_weight=None, dual=False,

In [26]:
X_y_data, df_event = ranking_data(df[df.is1], [3], col_names)
X_train, y_train = X_y_data[:,:-1], X_y_data[:,-1]
X_y_data, df_event = ranking_data(df[df.oos], [3], col_names)
X_test, y_test = X_y_data[:,:-1], X_y_data[:,-1]

In [30]:
X_new_tr = np.zeros((X_train.shape[0], len(logits)))
X_new_ts = np.zeros((X_test.shape[0], len(logits)))
for i, clf in enumerate(logits):
    X_new_tr[:, i] = clf.predict_proba(X_train)[:,0]
    X_new_ts[:, i] = clf.predict_proba(X_test)[:,0]

In [39]:
X_new_tr.mean(axis =1).reshape(-1,1).shape

(12389L, 1L)

In [44]:
X_tr_log = np.log(X_new_tr.mean(axis =1).reshape(-1,1))
X_tr_log_1 = np.log(1-X_new_tr.mean(axis =1).reshape(-1,1))
X_ts_log = np.log(X_new_ts.mean(axis =1).reshape(-1,1))
X_ts_log_1 = np.log(1-X_new_ts.mean(axis =1).reshape(-1,1))

In [47]:
X_train_add = np.hstack((X_train, X_new_tr.mean(axis =1).reshape(-1,1)))
X_train_add = np.hstack((X_train_add, 1 -X_new_tr.mean(axis =1).reshape(-1,1)))
X_train_add = np.hstack((X_train_add, X_tr_log))
X_train_add = np.hstack((X_train_add, X_tr_log_1))
X_test_add = np.hstack((X_test, X_new_ts.mean(axis =1).reshape(-1,1)))
X_test_add = np.hstack((X_test_add, 1 -X_new_ts.mean(axis =1).reshape(-1,1)))
X_test_add = np.hstack((X_test_add, X_ts_log))
X_test_add = np.hstack((X_test_add, X_ts_log_1))

In [48]:
print logit.fit(X_train_add, y_train)
print 'train score  ',logit.score(X_train_add, y_train)
print 
print 'test score  ', accuracy_score(y_test, logit.predict(X_test_add))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
train score   0.57438049883

test score   0.556531284303


In [55]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C = 1.0, loss ='hinge')
print svm.fit(X_train_add, y_train)
print 'train score  ',svm.score(X_train_add, y_train)
print 
print 'test score  ', accuracy_score(y_test, svm.predict(X_test_add))

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)
train score   0.57090967794

test score   0.559385290889


In [56]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
print gbc.fit(X_train_add, y_train)
print 'train score  ',gbc.score(X_train_add, y_train)
print 
print 'test score  ', accuracy_score(y_test, gbc.predict(X_test_add))

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)
train score   0.643151182501

test score   0.550384193194


In [57]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100)
print abc.fit(X_train_add, y_train)
print 'train score  ',abc.score(X_train_add, y_train)
print 
print 'test score  ', accuracy_score(y_test, abc.predict(X_test_add))

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)
train score   0.61514246509

test score   0.547969264544


In [58]:
from sklearn.neighbors import KNeighborsClassifier
kMM = KNeighborsClassifier (n_neighbors =100, n_jobs =-1)
print kMM.fit(X_train_add, y_train)
print 'train score  ',kMM.score(X_train_add, y_train)
print 
print 'test score  ', accuracy_score(y_test, kMM.predict(X_test_add))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=100, p=2,
           weights='uniform')
train score   0.575349100008

test score   0.550164654226


In [61]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators =100, random_state = 7, oob_score=True, n_jobs =-1)
print forest.fit(X_train_add, y_train)
print 'train score  ',forest.oob_score_
print 
print 'test score  ', accuracy_score(y_test, forest.predict(X_test_add))

 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=True, random_state=7,
            verbose=0, warm_start=False)
train score   0.545161029946

test score   0.549506037322


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()
print LDA.fit(X_train_add, y_train)
print 
print 'train score  ',LDA.score(X_train_add, y_train)
print 
print 'test score  ', accuracy_score(y_test, LDA.predict(X_test_add))

In [64]:
mean = forest.predict_proba(X_test_add)[:,0] + logit.predict_proba(X_test_add)[:,0] +\
gbc.predict_proba(X_test_add)[:,0] + abc.predict_proba(X_test_add)[:,0]

In [69]:
accuracy_score(y_test, np.where(mean/4 >= 0.5, -1, 1))

0.55828759604829858

- дополнительные признаки квадраты факторов

In [14]:
col_names = ['f%s'%i for i in range(1,58)]+['2f%s'%i for i in range(1,58)]
df2 = pd.DataFrame(data= np.hstack((df.ix[:,u'f1':u'f57'].values, df.ix[:,u'f1':u'f57'].values*df.ix[:,u'f1':u'f57'].values)), 
              columns = col_names)
for col in ['result', 'event_id', 'is1', 'oos']:
    df2[col] =df[col]
df2.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,2f52,2f53,2f54,2f55,2f56,2f57,result,event_id,is1,oos
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,...,0.115769,0.382722,0.001619,0.000012,0.000011,0.001353,3,287398,True,False
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,...,0.028750,0.160713,0.032101,0.000008,0.000008,0.045372,5,287398,True,False
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,...,0.097088,0.184932,0.047268,0.000150,0.000147,0.029932,13,287398,True,False
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,...,0.020284,0.302158,0.159399,0.000096,0.000098,0.106788,1,287398,True,False
4,0.064913,1.343377,0.242782,0.008894,0.0007,-0.017472,-0.199782,0.004808,0.481565,1.404777,...,0.056028,0.053604,0.105756,0.000022,0.000023,0.021267,6,287398,True,False


In [ ]:
clf_llhood (df2, logit, result =[2], columns = col_names +['result','event_id'])

In [18]:
X_y_data, df_event = ranking_data(df[df.is1], [4])
X_train, y_train = X_y_data[:,:-1], X_y_data[:,-1]
X_y_data, df_event = ranking_data(df[df.oos], [4])
X_test, y_test = X_y_data[:,:-1], X_y_data[:,-1]

In [19]:
logit.predict_log_proba(X_train)[:,:1].shape, X_train.shape

((12081L, 1L), (12081L, 57L))

In [20]:
X_tr_log_p  = np.hstack((X_train, logit.predict_log_proba(X_train)[:,:1]))
X_ts_log_p  = np.hstack((X_test, logit.predict_log_proba(X_test)[:,:1]))

In [21]:
logit1 = LogisticRegression (C =1., penalty ='l2', n_jobs =-1, class_weight ='balanced')
logit1.fit(X_tr_log_p, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [22]:
logit1.score(X_tr_log_p, y_train)

0.5569903153712441

In [23]:
accuracy_score(y_test, logit1.predict(X_ts_log_p))

0.55380813300381937

In [24]:
clf_llhood(df, logit, [2])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
accuracy   0.590998902305

is1  mean ll  -2610.25367312

oos  mean ll   -2661.45364861


In [25]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C = 1.0 )
clf_llhood(df, svm, [2])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
accuracy   0.591877058178

is1  mean ll  -2396.41145935

oos  mean ll   -2435.96723734


In [26]:
S_train = X_train.dot(logit.coef_[0])
S_test = X_test.dot(logit.coef_[0])

In [30]:
S_max = np.abs(S_train).max()
np.abs(S_train).max(), np.abs(S_train).min()

(4.400385377827603, 0.00012600539310712866)

In [31]:
X_train_S = X_train*(S_max/ S_train).reshape(-1,1)
X_test_S = X_test*(S_max/ S_test).reshape(-1,1)

In [32]:
logit1.fit(X_train_S, y_train)
logit1.score(X_train_S, y_train)

0.51030543829153219

In [33]:
accuracy_score(y_test, logit1.predict(X_test_S))

0.49494495618962031

In [34]:
X_y_data, df_event = ranking_data(df[df.is1], [2])
X_train, y_train = X_y_data[:,:-1], X_y_data[:,-1]
X_y_data, df_event_ts = ranking_data(df[df.oos], [2])
X_test, y_test = X_y_data[:,:-1], X_y_data[:,-1]

In [35]:
from sklearn.mixture import BayesianGaussianMixture
BGM = BayesianGaussianMixture(n_components=6 )

In [81]:
BGM.fit(X_train, y_train)

BayesianGaussianMixture(covariance_prior=None, covariance_type='full',
            degrees_of_freedom_prior=None, init_params='kmeans',
            max_iter=100, mean_precision_prior=None, mean_prior=None,
            n_components=6, n_init=1, random_state=None, reg_covar=1e-06,
            tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
            weight_concentration_prior=None,
            weight_concentration_prior_type='dirichlet_process')

In [100]:
cl = BGM.predict(X_train)

In [101]:
cluster_P_train= BGM.predict_proba(X_train)
cluster_P_test= BGM.predict_proba(X_test)

In [102]:
df_cluster = pd.DataFrame(data = BGM.predict(X_train), index = df_event.values, columns =[['cl']])

In [103]:
for n in np.unique(df_cluster['cl']):
    df_cluster['cl_%s'%n] = cluster_P_train[:, n]

In [104]:
df_cluster.head()

,cl,cl_0,cl_1,cl_2,cl_3,cl_4,cl_5
287398,1,1.552685e-99,1.000000,3.280995e-09,4.502352e-271,0.0,3.840932e-10
287399,1,1.277027e-185,1.000000,1.165435e-09,0.000000e+00,0.0,5.345455e-10
287400,1,4.148036e-49,1.000000,9.510896e-10,0.000000e+00,0.0,6.766541e-08
287401,2,1.452970e-05,0.000000,9.999855e-01,0.000000e+00,0.0,4.725975e-23
287402,1,1.100479e-18,0.999999,6.510512e-07,9.626145e-291,0.0,2.947819e-13


In [105]:
df_cluster_ts = pd.DataFrame(data = BGM.predict(X_test), index = df_event_ts.values, columns =[['cl']])

In [106]:
for n in np.unique(df_cluster_ts['cl']):
    df_cluster_ts['cl_%s'%n] = cluster_P_test[:, n]

In [107]:
cl_logit =[]
for n in np.unique(df_cluster['cl']):
    
    logit1 = LogisticRegression (C =1., penalty ='l2', n_jobs =-1, class_weight ='balanced')
    X_tr_cl = X_train[(df_cluster['cl'] == n).values]
    y_tr_cl = y_train[(df_cluster['cl'] == n).values]
    print 'cluster  %s,  count %s'%(n,len(y_tr_cl))
    print logit1.fit(X_tr_cl, y_tr_cl)
    print 'train score  ',logit1.score(X_tr_cl, y_tr_cl)
    print
    cl_logit.append(logit1)

cluster  0,  count 3732
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
train score   0.595391211147

cluster  1,  count 5563
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
train score   0.582060039547

cluster  2,  count 1426
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
train score   0.654978962132

cluster  3,  count 20


In [108]:
proba = np.zeros((X_train.shape[0], 2))
for n in np.unique(df_cluster['cl']): 
    proba = proba +cl_logit[n].predict_proba(X_train)* df_cluster['cl_%s'%n].values.reshape(-1,1)

In [109]:
proba_ts = np.zeros((X_test.shape[0], 2))
for n in np.unique(df_cluster_ts['cl']): 
    proba_ts = proba_ts +cl_logit[n].predict_proba(X_test)* df_cluster_ts['cl_%s'%n].values.reshape(-1,1)

In [110]:
predict_tr = np.ones((len(proba)))
predict_tr = np.where(np.argmax(proba, axis = 1) == 0, -1, 1)
1.*((y_train - predict_tr) == 0).sum()/len(y_train)

0.5966583259342966

In [111]:
predict_ts = np.ones((len(proba_ts)))
predict_ts = np.where(np.argmax(proba_ts, axis = 1) == 0, -1, 1)
1.*((y_test - predict_ts) == 0).sum()/len(y_test)

0.5815587266739847

In [112]:
df_cluster_add = df_cluster.append(df_cluster_ts)
df_cluster_add.shape

(16944, 7)

In [113]:
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,runner_id,result,event_id,is1,is2,oos,S,p,log_p,cl_x,cl_0_x,cl_1_x,cl_2_x,cl_3_x,cl_4_x,cl_5_x,S_cl_0,p_cl__0,S_cl_1,p_cl__1,S_cl_2,p_cl__2,S_cl_3,p_cl__3,S_cl_4,p_cl__4,S_cl_5,p_cl__5,p_mix,cl_y,cl_0_y,cl_1_y,cl_2_y,cl_3_y,cl_4_y,cl_5_y
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,0.374753,0.675330,0.051377,-0.005242,1.305105,0.083228,-0.003275,0.032186,0.00289,0.041683,0.015889,-0.358249,0.287939,0.037986,-0.008733,-0.001490,-0.472996,-0.003019,-0.010773,-0.027652,-0.123439,0.040229,0.109841,0.000775,-0.002725,-0.215236,-0.173222,0.859616,-0.003327,1.886817,0.450763,0.508958,1.094065,-0.001936,0.065516,0.386516,0.049406,0.002729,0.490221,0.226400,-0.006899,-0.340249,0.618645,0.040241,-0.003450,-0.003327,0.036781,353046,3,287398,True,True,False,-0.331537,0.053286,-3.310580,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.678621,0.032506,-0.667135,0.032394,-0.861541,0.028221,-1.299192,0.011225,0.351216,0.088811,-0.596701,0.036495,0.036495,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,-0.994794,0.212441,0.484803,0.010791,1.408825,0.066311,-0.002738,-0.017447,0.00289,-0.012072,-0.023179,0.047898,-0.533036,-0.086651,-0.008733,0.000512,0.075571,0.000948,-0.044777,-0.020852,0.433588,-0.145489,0.176984,0.000775,-0.002182,0.030317,0.034727,1.008091,-0.002790,1.810774,0.289262,0.620846,0.491455,0.137752,-0.020472,0.191353,-0.296367,-0.051450,-0.250389,-0.015683,-0.006899,0.169559,-0.400890,0.179168,-0.002913,-0.002791,0.213007,352217,5,287398,True,True,False,-0.213620,0.059955,-3.546679,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.579957,0.035877,-0.805975,0.028195,-0.281796,0.050391,-1.325842,0.010930,1.418878,0.258312,-0.832801,0.028820,0.028820,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,0.135928,0.519989,0.158056,0.013730,0.722187,0.170334,-0.012058,-0.011141,0.00289,-0.013402,-0.022607,-0.209299,-0.519305,0.005360,-0.008733,0.001372,-0.251839,0.002045,0.048360,-0.034847,-0.469892,-0.054636,-0.129261,0.000775,-0.011611,-0.194838,-0.143116,0.357578,-0.012107,1.689672,-0.294596,0.774879,0.741813,0.082078,-0.047917,0.386516,-0.296367,0.002728,0.583518,0.034111,-0.006899,-0.311590,0.430037,0.217412,-0.012249,-0.012108,-0.173009,342962,13,287398,True,True,False,-0.094424,0.067545,-2.829194,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.152950,0.054987,-0.368198,0.043681,-0.167563,0.056489,-0.195104,0.033859,-0.124884,0.055169,-0.115315,0.059060,0.059060,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,0.374753,0.212441,0.584892,0.025233,1.098121,0.119076,0.009932,0.020942,0.00289,-0.012072,0.015889,0.062337,0.553817,0.053025,-0.008733,-0.003080,0.018702,-0.002474,-0.010773,0.086458,0.020163,0.200410,0.091943,0.000775,0.010607,-0.073015,-0.061893,0.557530,0.009877,1.360255,0.365248,-0.333936,0.307492,-0.008507,-0.056499,0.191353,0.074037,-0.007525,0.490221,0.226400,-0.006899,-0.142420,0.549689,0.399248,0.009783,0.009877,0.326784,361432,1,287398,True,True,False,-0.256826,0.057420,-3.334780,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.460113,0.040445,-0.512950,0.037794,-0.503278,0.040380,-1.607062,0.008250,0.510964,0.104194,-0.620902,0.035622,0.035622,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
4,0.064913,1.343377,0.242782,0.008894,0.0007,-0.017472,-0.199782,0.004808,0.481565,1.404777,0.326130,0.248326,-0.371251,-0.005610,1.051047,-0.331326,0.004842,0.043404,0

In [114]:
df = df.merge(df_cluster_add, left_on = df['event_id'], right_index = True)
pd.set_option('display.max_columns', 100)
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,runner_id,result,event_id,is1,is2,oos,S,p,log_p,cl_x,cl_0_x,cl_1_x,cl_2_x,cl_3_x,cl_4_x,cl_5_x,S_cl_0,p_cl__0,S_cl_1,p_cl__1,S_cl_2,p_cl__2,S_cl_3,p_cl__3,S_cl_4,p_cl__4,S_cl_5,p_cl__5,p_mix,cl_y,cl_0_y,cl_1_y,cl_2_y,cl_3_y,cl_4_y,cl_5_y,cl,cl_0,cl_1,cl_2,cl_3,cl_4,cl_5
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,0.374753,0.675330,0.051377,-0.005242,1.305105,0.083228,-0.003275,0.032186,0.00289,0.041683,0.015889,-0.358249,0.287939,0.037986,-0.008733,-0.001490,-0.472996,-0.003019,-0.010773,-0.027652,-0.123439,0.040229,0.109841,0.000775,-0.002725,-0.215236,-0.173222,0.859616,-0.003327,1.886817,0.450763,0.508958,1.094065,-0.001936,0.065516,0.386516,0.049406,0.002729,0.490221,0.226400,-0.006899,-0.340249,0.618645,0.040241,-0.003450,-0.003327,0.036781,353046,3,287398,True,True,False,-0.331537,0.053286,-3.310580,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.678621,0.032506,-0.667135,0.032394,-0.861541,0.028221,-1.299192,0.011225,0.351216,0.088811,-0.596701,0.036495,0.036495,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,-0.994794,0.212441,0.484803,0.010791,1.408825,0.066311,-0.002738,-0.017447,0.00289,-0.012072,-0.023179,0.047898,-0.533036,-0.086651,-0.008733,0.000512,0.075571,0.000948,-0.044777,-0.020852,0.433588,-0.145489,0.176984,0.000775,-0.002182,0.030317,0.034727,1.008091,-0.002790,1.810774,0.289262,0.620846,0.491455,0.137752,-0.020472,0.191353,-0.296367,-0.051450,-0.250389,-0.015683,-0.006899,0.169559,-0.400890,0.179168,-0.002913,-0.002791,0.213007,352217,5,287398,True,True,False,-0.213620,0.059955,-3.546679,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.579957,0.035877,-0.805975,0.028195,-0.281796,0.050391,-1.325842,0.010930,1.418878,0.258312,-0.832801,0.028820,0.028820,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,0.135928,0.519989,0.158056,0.013730,0.722187,0.170334,-0.012058,-0.011141,0.00289,-0.013402,-0.022607,-0.209299,-0.519305,0.005360,-0.008733,0.001372,-0.251839,0.002045,0.048360,-0.034847,-0.469892,-0.054636,-0.129261,0.000775,-0.011611,-0.194838,-0.143116,0.357578,-0.012107,1.689672,-0.294596,0.774879,0.741813,0.082078,-0.047917,0.386516,-0.296367,0.002728,0.583518,0.034111,-0.006899,-0.311590,0.430037,0.217412,-0.012249,-0.012108,-0.173009,342962,13,287398,True,True,False,-0.094424,0.067545,-2.829194,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.152950,0.054987,-0.368198,0.043681,-0.167563,0.056489,-0.195104,0.033859,-0.124884,0.055169,-0.115315,0.059060,0.059060,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,0.374753,0.212441,0.584892,0.025233,1.098121,0.119076,0.009932,0.020942,0.00289,-0.012072,0.015889,0.062337,0.553817,0.053025,-0.008733,-0.003080,0.018702,-0.002474,-0.010773,0.086458,0.020163,0.200410,0.091943,0.000775,0.010607,-0.073015,-0.061893,0.557530,0.009877,1.360255,0.365248,-0.333936,0.307492,-0.008507,-0.056499,0.191353,0.074037,-0.007525,0.490221,0.226400,-0.006899,-0.142420,0.549689,0.399248,0.009783,0.009877,0.326784,361432,1,287398,True,True,False,-0.256826,0.057420,-3.334780,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.460113,0.040445,-0.512950,0.037794,-0.503278,0.040380,-1.607062,0.008250,0.510964,0.104194,-0.620902,0.035622,0.035622,1,1.55268

In [115]:
factors = ['f{}'.format(i) for i in range(1,58)]
for n in np.unique(df['cl']):
        
    df['S_cl_%s'%n] = np.dot(df[factors].values ,cl_logit[n].coef_[0].reshape(-1,1))
    df['p_cl__%s'%n] = df['S_cl_%s'%n].groupby(by = df.event_id).apply(softmax)
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,runner_id,result,event_id,is1,is2,oos,S,p,log_p,cl_x,cl_0_x,cl_1_x,cl_2_x,cl_3_x,cl_4_x,cl_5_x,S_cl_0,p_cl__0,S_cl_1,p_cl__1,S_cl_2,p_cl__2,S_cl_3,p_cl__3,S_cl_4,p_cl__4,S_cl_5,p_cl__5,p_mix,cl_y,cl_0_y,cl_1_y,cl_2_y,cl_3_y,cl_4_y,cl_5_y,cl,cl_0,cl_1,cl_2,cl_3,cl_4,cl_5
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,0.374753,0.675330,0.051377,-0.005242,1.305105,0.083228,-0.003275,0.032186,0.00289,0.041683,0.015889,-0.358249,0.287939,0.037986,-0.008733,-0.001490,-0.472996,-0.003019,-0.010773,-0.027652,-0.123439,0.040229,0.109841,0.000775,-0.002725,-0.215236,-0.173222,0.859616,-0.003327,1.886817,0.450763,0.508958,1.094065,-0.001936,0.065516,0.386516,0.049406,0.002729,0.490221,0.226400,-0.006899,-0.340249,0.618645,0.040241,-0.003450,-0.003327,0.036781,353046,3,287398,True,True,False,-0.331537,0.053286,-3.310580,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.766702,0.022774,-0.864577,0.024444,-0.709182,0.031982,-1.164725,0.013549,-2.072146,0.005716,-0.615573,0.032584,0.036495,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,-0.994794,0.212441,0.484803,0.010791,1.408825,0.066311,-0.002738,-0.017447,0.00289,-0.012072,-0.023179,0.047898,-0.533036,-0.086651,-0.008733,0.000512,0.075571,0.000948,-0.044777,-0.020852,0.433588,-0.145489,0.176984,0.000775,-0.002182,0.030317,0.034727,1.008091,-0.002790,1.810774,0.289262,0.620846,0.491455,0.137752,-0.020472,0.191353,-0.296367,-0.051450,-0.250389,-0.015683,-0.006899,0.169559,-0.400890,0.179168,-0.002913,-0.002791,0.213007,352217,5,287398,True,True,False,-0.213620,0.059955,-3.546679,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.714238,0.024001,-0.536705,0.033928,-0.275155,0.049363,0.008524,0.043798,-1.809029,0.007437,-1.079423,0.020491,0.028820,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,0.135928,0.519989,0.158056,0.013730,0.722187,0.170334,-0.012058,-0.011141,0.00289,-0.013402,-0.022607,-0.209299,-0.519305,0.005360,-0.008733,0.001372,-0.251839,0.002045,0.048360,-0.034847,-0.469892,-0.054636,-0.129261,0.000775,-0.011611,-0.194838,-0.143116,0.357578,-0.012107,1.689672,-0.294596,0.774879,0.741813,0.082078,-0.047917,0.386516,-0.296367,0.002728,0.583518,0.034111,-0.006899,-0.311590,0.430037,0.217412,-0.012249,-0.012108,-0.173009,342962,13,287398,True,True,False,-0.094424,0.067545,-2.829194,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.262025,0.037724,-0.273273,0.044154,-0.016487,0.063935,-0.810425,0.019310,-1.342198,0.011861,-0.488618,0.036995,0.059060,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10,1,1.552685e-99,1.0,3.280995e-09,4.502352e-271,0.0,3.840932e-10
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,0.374753,0.212441,0.584892,0.025233,1.098121,0.119076,0.009932,0.020942,0.00289,-0.012072,0.015889,0.062337,0.553817,0.053025,-0.008733,-0.003080,0.018702,-0.002474,-0.010773,0.086458,0.020163,0.200410,0.091943,0.000775,0.010607,-0.073015,-0.061893,0.557530,0.009877,1.360255,0.365248,-0.333936,0.307492,-0.008507,-0.056499,0.191353,0.074037,-0.007525,0.490221,0.226400,-0.006899,-0.142420,0.549689,0.399248,0.009783,0.009877,0.326784,361432,1,287398,True,True,False,-0.256826,0.057420,-3.334780,5,2.583198e-76,1.217674e-16,1.656811e-08,0.0,6.907297e-283,1.0,-0.652312,0.025534,-0.721107,0.028215,-0.435250,0.042060,-1.040604,0.015340,-0.852562,0.019354,-0.442834,0.038728,0.035622,1,1.552

In [116]:
prob_mix = np.zeros((len(df)))
df.fillna(0, inplace =True)
for n in np.unique(df['cl']):
    
    prob_mix = prob_mix + df['cl_%s'%n]*df['p_cl__%s'%n]
df['p_mix'] = prob_mix
df.tail()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,runner_id,result,event_id,is1,is2,oos,S,p,log_p,cl_x,cl_0_x,cl_1_x,cl_2_x,cl_3_x,cl_4_x,cl_5_x,S_cl_0,p_cl__0,S_cl_1,p_cl__1,S_cl_2,p_cl__2,S_cl_3,p_cl__3,S_cl_4,p_cl__4,S_cl_5,p_cl__5,p_mix,cl_y,cl_0_y,cl_1_y,cl_2_y,cl_3_y,cl_4_y,cl_5_y,cl,cl_0,cl_1,cl_2,cl_3,cl_4,cl_5
174298,-0.321080,0.263185,0.116496,0.013721,0.001686,0.098474,0.153590,-0.035127,0.136799,-0.028383,0.400107,0.116757,-0.209353,-0.064880,-0.085903,0.019455,-0.034992,-0.062260,-0.022169,0.028282,-0.050458,-0.183807,-0.217550,-0.143204,-0.010932,0.001542,-0.335943,-0.007364,-0.101062,-0.014607,-0.570272,-0.123502,-0.648533,-0.002281,-0.035335,0.046689,0.085554,0.022733,-0.035128,0.215129,-0.502835,0.681566,0.045806,-0.051037,0.056224,-0.034666,-0.132721,0.051436,-0.354936,0.436300,-0.004281,-0.107041,0.359159,-0.051952,-0.035207,-0.035137,-0.032829,448142,2,337025,False,False,True,-0.021203,0.194703,-1.930212,0,0.999537,6.884783e-07,0.000459,0.0,0.0,0.000003,-0.291616,0.136682,0.321453,0.266420,0.018743,0.192890,0.013695,0.186395,-0.044865,0.186884,-0.236799,0.156000,0.136683,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003
174299,-0.014688,0.361564,-0.151997,-0.008917,0.001686,0.128687,0.047174,0.016257,0.208812,0.229246,0.154541,0.101702,0.111508,0.026485,0.820248,-0.012151,0.016400,0.033626,0.018828,-0.009580,-0.022102,0.398254,-0.124395,0.020805,-0.010932,-0.001166,0.159201,0.000145,0.015720,-0.020251,-0.186409,-0.063011,-0.119374,-0.002281,0.016662,-0.164145,-0.224057,0.784832,0.016252,0.197853,0.036053,-0.687208,0.145379,0.045346,-0.024538,0.070196,-0.132721,-0.063144,0.454167,0.532934,-0.004281,-0.298923,0.352397,-0.012849,0.016282,0.016245,-0.009948,492119,3,337025,False,False,True,-0.054315,0.188361,-1.783385,0,0.999537,6.884783e-07,0.000459,0.0,0.0,0.000003,-0.252755,0.142099,-0.306361,0.142204,0.059433,0.200900,-0.621389,0.098769,0.303505,0.264769,-0.036182,0.190656,0.142099,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003
174300,0.365143,0.361564,-0.086240,0.006008,-0.007694,-0.039630,0.047174,0.011645,0.095136,0.199507,0.085536,-0.174135,0.125581,0.008427,0.211824,0.029606,0.010887,0.025491,0.007185,-0.005459,0.076477,0.301935,0.190694,0.054606,0.089622,-0.000850,0.274039,0.001102,0.084583,0.139013,-0.181795,-0.044473,0.058390,0.028426,0.012864,-0.093669,-0.141739,0.140447,0.011719,-0.025098,0.807525,0.082426,-0.079214,0.030437,-0.002493,-0.024679,-0.132721,0.091730,0.370001,0.513737,0.029828,-0.185056,-0.036075,0.032794,0.013120,0.011713,0.002146,493020,1,337025,False,False,True,-0.019709,0.194994,-1.686988,0,0.999537,6.884783e-07,0.000459,0.0,0.0,0.000003,-0.125493,0.161383,-0.161143,0.164429,0.004230,0.190111,-0.291874,0.137318,-0.078714,0.180664,-0.056763,0.186773,0.161384,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003
174301,-0.014688,0.227617,0.019758,-0.008917,0.001686,-0.074838,-0.123969,0.016257,-0.075395,-0.136683,-0.436639,-0.058350,0.042018,0.026485,-0.040536,-0.054279,0.016400,-0.004008,0.018828,-0.009580,0.047789,0.138720,0.876229,0.013187,-0.010932,-0.002542,0.161147,0.002912,0.015720,0.002668,0.708288,0.325818,0.425668,-0.002281,0.016662,0.561480,0.210187,-0.002552,0.016252,-0.059128,-0.051544,-0.159210,0.022945,0.021892,-0.024538,0.027447,0.530884,-0.063144,-0.168643,-0.568762,-0.004281,0.641797,-0.621707,0.006831,0.016282,0.016245,0.011049,492127,4,337025,False,False,True,-0.103867,0.179255,-1.822821,0,0.999537,6.884783e-07,0.000459,0.0,0.0,0.000003,-0.096432,0.166142,-0.159816,0.164647,-0.573824,0.106650,0.392566,0.272255,0.150692,0.227249,0.203261,0.242237,0.166142,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003,0,0.999993,0.000001,0.000003,0.0,0.0,0.000003
174302,-0.014688,-1.213931,0.

In [117]:
df_result = df.groupby(df.event_id).apply(lambda x: x.result[x.p_mix.argmax()])

In [118]:
((df_result ==1)|(df_result ==2)|(df_result ==3)).sum()

2589

In [119]:
df['log_p'] = np.log(df['p_mix'])
print 'll is1',df.ix[df.is1 & (df.result ==1),'log_p'].mean()*1000
print 'll oos',df.ix[df.oos & (df.result ==1),'log_p'].mean()*1000

ll is1 -2769.0101766
ll oos -2759.33303157


In [120]:
from sklearn.decomposition import PCA
pca = PCA(n_components =2)
cl_pca =[]
cl_logit_pca = []
for n in np.unique(df_cluster['cl']):
    
    logit1 = LogisticRegression (C =1., penalty ='l2', n_jobs =-1, class_weight ='balanced')
    X_tr_cl = X_train[(df_cluster['cl'] == n).values]
    y_tr_cl = y_train[(df_cluster['cl'] == n).values]
    X_new, pca = PCA_features(X_tr_cl, pca)
    
    print 'cluster  %s,  count %s'%(n,len(y_tr_cl))
    print logit1.fit(X_new, y_tr_cl)
    print 'train score  ',logit1.score(X_new, y_tr_cl)
    print
    cl_pca.append(pca)
    cl_logit_pca.append(logit1)

cluster  0,  count 3732
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
train score   0.60691318328

cluster  1,  count 5563
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
train score   0.596620528492

cluster  2,  count 1426
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
train score   0.672510518934

cluster  3,  count 20
L

In [121]:
from itertools import combinations
for n,k in combinations([0, 1, 2, 5], 2):
    diff_coef = np.abs(cl_logit_pca[n].coef_[0] -cl_logit_pca[k].coef_[0])
    print '{} -{} clustr   max diff {} coef {}'.format(n,k,diff_coef.argmax(), diff_coef.max())

0 -1 clustr   max diff 133 coef 1.51116303658
0 -2 clustr   max diff 82 coef 1.28600879329
0 -5 clustr   max diff 118 coef 0.969379582651
1 -2 clustr   max diff 131 coef 1.26791787489
1 -5 clustr   max diff 2 coef 1.74322336122
2 -5 clustr   max diff 13 coef 1.43939797241


In [122]:
X_y_data, df_event = ranking_data(df[df.is1], [2])
X_train, y_train = X_y_data[:,:-1], X_y_data[:,-1]
X_y_data, df_event_ts = ranking_data(df[df.oos], [2])
X_test, y_test = X_y_data[:,:-1], X_y_data[:,-1]

In [123]:
from sklearn.preprocessing import PolynomialFeatures

In [124]:
poly = PolynomialFeatures()
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

In [126]:
X_poly_train.shape, X_poly_test.shape

((12389L, 1711L), (4555L, 1711L))

In [128]:
logit_p = logit1 = LogisticRegression (C =1., penalty ='l2', n_jobs =-1)

In [129]:
logit_p.fit(X_poly_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [130]:
logit_p.score(X_poly_train, y_train)

0.64799418839292922

In [131]:
accuracy_score(y_test, logit_p.predict(X_poly_test))

0.54753018660812292

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier()

In [28]:
%time GBC.fit(X_train, y_train)

Wall time: 51 s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [29]:
gbc_acc = accuracy_score(y_test, GBC.predict(X_test))
gbc_acc

0.57252415013642066

In [30]:
GBC.decision_function(X_test).shape

(13561L,)

In [31]:
df['S_gbc'] = GBC.decision_function(df[factors].values)
df['p_gbc'] = df['S_gbc'].groupby(by = df.event_id).apply(softmax)
df['log_p_gbc'] = np.log(df['p_gbc'])
print 'll is1',df.ix[df.is1 & (df.result ==1),'log_p_gbc'].mean()*1000
print 'll oos',df.ix[df.oos & (df.result ==1),'log_p_gbc'].mean()*1000

ll is1 -2529.42053198
ll oos -2572.45382418


In [32]:
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,oos,S,p,log_p,S_pca,p_pca,log_p_pca,S_gbc,p_gbc,log_p_gbc
0,0.064913,1.395654,-0.152068,0.008172,0.0007,0.971937,1.096287,-0.003309,1.761018,0.968297,...,False,-0.563592,0.039677,-3.226976,-0.518235,0.041320,-3.186417,-0.634022,0.038235,-3.264015
1,-0.250759,1.137530,0.019747,0.008894,0.0007,0.398019,0.481980,-0.002772,1.682896,0.354919,...,False,-0.298126,0.051741,-2.961511,-0.314351,0.050664,-2.982533,-0.367064,0.049934,-2.997057
2,-0.250759,1.029570,-0.145221,-0.020776,0.0007,0.307072,0.427015,-0.012089,1.165470,0.440719,...,False,-0.199542,0.057101,-2.862927,-0.149040,0.059772,-2.817222,-0.279891,0.054482,-2.909885
3,0.064913,1.395654,-0.136884,-0.020776,0.0007,0.234280,0.361007,0.009897,0.880806,1.230350,...,False,-0.397436,0.046849,-3.060820,-0.416026,0.045766,-3.084208,-0.418661,0.047423,-3.048654
4,0.064913,1.343377,0.242782,0.008894,0.0007,-0.017472,-0.199782,0.004808,0.481565,1.404777,...,False,-0.321414,0.050550,-2.984798,-0.297887,0.051505,-2.966069,-0.490446,0.044138,-3.120440


In [61]:
columns = ['f{}'.format(i) for i in range(1,58)]+ ['log_p_mean'] +['result','event_id']
#columns
from sklearn.svm import LinearSVC
svm = LinearSVC(C = 0.10 )
clf_llhood(df, svm, [2, 3, 4], columns)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
accuracy   0.573040336258

is1  mean ll  -2372.27720991

oos  mean ll   -2409.29576695


In [47]:
np.argsort(np.abs(GBC.feature_importances_), order =)

array([38, 45,  7, 34, 55, 50, 16, 54, 20, 48, 41, 19, 12, 24, 23,  4, 46,
       32, 26, 40, 28, 18, 31,  2, 33, 36, 17,  9, 27, 51,  0, 13, 43,  5,
       25, 11, 42, 52, 15, 35, 37, 47, 53, 14,  3, 30, 10, 56, 22, 21, 44,
       29, 39,  8, 49,  6,  1], dtype=int64)

In [43]:
GBC.feature_importances_[38], GBC.feature_importances_[1]

(0.00084886972704853166, 0.053257303242747057)

In [48]:
np.argsort(np.abs(logit.coef_[0]))

array([12, 28,  1, 30, 56, 41, 42, 51,  5, 45, 21, 32,  9, 22, 39, 14, 23,
        4, 50, 35, 11, 48, 10, 54, 38, 55,  7, 17, 16, 25, 26, 13, 40, 53,
       20, 52, 31,  6, 18, 24,  8, 34, 15,  0, 49, 27, 37, 46, 29, 43, 47,
       19,  3, 36,  2, 33, 44], dtype=int64)

In [50]:
logit.coef_[0][12], logit.coef_[0][44], logit.coef_[0][33]

(-0.0016302758853066514, -0.37900786531156072, 0.28502828870319247)

In [52]:
X_train_new = np.hstack((X_train ,X_train* X_train[:,44].reshape(-1,1)))
X_test_new = np.hstack((X_test, X_test* X_test[:,44].reshape(-1,1)))

In [53]:
logit2.fit(X_train_new, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [54]:
accuracy_score(y_test, logit2.predict(X_test_new))

0.56861588378438166

In [59]:
df['S_sum'] = df['p'] +df['p_gbc']
df['p_mean'] = df['S_sum'].groupby(by = df.event_id).apply(softmax)
df['log_p_mean'] = np.log(df['p_mean'])
print 'll is1',df.ix[df.is1 & (df.result ==1),'log_p_mean'].mean()*1000
print 'll oos',df.ix[df.oos & (df.result ==1),'log_p_mean'].mean()*1000

ll is1 -2289.44713932
ll oos -2321.91678668


In [39]:
from sklearn.svm import SVC
SVM = SVC()

In [40]:
%time SVM.fit(X_train, y_train)

Wall time: 4min 20s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [41]:
SVM_acc = accuracy_score(y_test, SVM.predict(X_test))
SVM_acc

0.57215544576358679

In [42]:
from sklearn.ensemble import ExtraTreesClassifier
ERT = ExtraTreesClassifier()

In [43]:
%time ERT.fit(X_train, y_train)

Wall time: 1.83 s


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [44]:
ERT_acc = accuracy_score(y_test, ERT.predict(X_test))
ERT_acc

0.52909077501659174

In [45]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100)

In [46]:
%time abc.fit(X_train, y_train)

Wall time: 48.8 s


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [47]:
abc_acc = accuracy_score(y_test, abc.predict(X_test))
abc_acc

0.56596121229997787

In [48]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()

In [49]:
%time clf.fit(X_train, y_train)

Wall time: 42.5 s


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [50]:
clf_acc = accuracy_score(y_test, clf.predict(X_test))
clf_acc

0.53063933338249392

In [55]:
clf = MLPClassifier(hidden_layer_sizes=(50, 20))
%time clf.fit(X_train, y_train)

Wall time: 51.2 s


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [56]:
clf_acc = accuracy_score(y_test, clf.predict(X_test))
clf_acc

0.52370769117321736

In [21]:

n =2
df_runner = df[['event_id', 'runner_id']][df['result'] == n-1]
for n in [2, 3, 4]:
    df_runner2 = df[['event_id', 'runner_id']][df['result'] == n]
    df_runner = pd.merge(df_runner, df_runner2, how = 'left', left_on = 'event_id', right_on = 'event_id' ,
                         suffixes =('_%s'%(n-1), '_%s'%n))
df_runner.head()

,event_id,runner_id_1,runner_id_2,runner_id_3,runner_id_4
0,287398,361432,355937,353046,363590.0
1,287399,349555,346146,358422,363600.0
2,287400,288738,145512,164898,165432.0
3,287401,347129,302596,277210,323071.0
4,287402,356489,363613,313967,343102.0


In [18]:
def matrix_parwise(X_matrix):
    """
    return the matrix with data in each row is the differane between one choice row and other 
    <X_matrix> matrix where last column has runner_id and others columns have data
    """
    if not type(X_matrix) is np.ndarray:
        X_matrix = X_matrix.values
    one = np.random.choice(X_matrix[:,-1])
    mask_one = np.in1d(X_matrix[:,-1], one)
    X_one = X_matrix[np.logical_not(mask_one)]
    X_one[:,:-1] = X_one[:,:-1] - X_matrix[mask_one, :-1]
    #print X_one.shape, np.repeat(one, len(X_one)).reshape(-1,1).shape
    col_names = ['f{}'.format(i) for i in range(1,58)] + ['', 'runner_2', 'runner_1']
    df_ = pd.DataFrame(data = np.hstack((X_one, np.repeat(one, len(X_one)).reshape(-1,1))), columns= col_names )
    return df_.drop('', axis =1)

In [19]:
factors = ['f{}'.format(i) for i in range(1,58)]
df_parwise = df[factors +['event_id', 'runner_id']].groupby('event_id').apply(matrix_parwise)

In [20]:
df_parwise.head()

f1        f2        f3        f4        f5        f6  \
event_id                                                                 
287398   0  0.315672  0.366084 -0.006847  0.028947  0.000000  0.664865   
         1  0.000000  0.107960  0.164968  0.029670  0.000000  0.090947   
         2  0.315672  0.366084  0.008337  0.000000  0.000000 -0.072792   
         3  0.315672  0.313807  0.388003  0.029670  0.000000 -0.324543   
         4  0.433328  0.000000  0.346631  0.024042 -0.002916 -0.283627   

                  f7        f8        f9       f10    ...          f50  \
event_id                                              ...                
287398   0  0.669272  0.008781  0.595547  0.527577    ...     0.192288   
         1  0.054965  0.009317  0.517426 -0.085800    ...    -0.049795   
         2 -0.066008  0.021986 -0.284664  0.789631    ...     0.192288   
         3 -0.626797  0.016897 -0.683905  0.964058    ...     0.631435   
         4 -0.652280  0.013406 -1.003390 -0.319131    ...     0.137875   

                 f51       f52       f53       f54       f55       f56  \
event_id                                                                 
287398   0  0.000000 -0.028659  0.188608 -0.177171  0.008799  0.008780   
         1  0.000000  0.481149 -0.830927 -0.038244  0.009336  0.009317   
         2  0.000000  0.169170  0.119652  0.181836  0.022032  0.021985   
         3  0.000000  0.548292 -0.198511 -0.542613  0.016932  0.016896   
         4  0.032719  0.834669 -0.774801 -0.193563  0.014596  0.013467   

                 f57  runner_2  runner_1  
event_id                                  
287398   0  0.209790  353046.0  342962.0  
         1  0.386017  352217.0  342962.0  
         2  0.499794  361432.0  342962.0  
         3  0.027179  359420.0  342962.0  
         4  0.181092  363590.0  342962.0  

[5 rows x 59 columns]